In [1]:
%matplotlib inline


# Structure minimization and constant temperature MD using ASE interface

This example is modified from the official `home page` and
`Constant temperature MD`_ to use the ASE interface of TorchANI as energy
calculator.

    https://wiki.fysik.dtu.dk/ase/
    https://wiki.fysik.dtu.dk/ase/tutorials/md/md.html#constant-temperature-md


To begin with, let's first import the modules we will use:



In [2]:
from ase.lattice.cubic import Diamond
from ase.md.langevin import Langevin
from ase.optimize import BFGS
from ase import units
import torchani

/Users/sergiortizropero/miniconda3/envs/ASE_ANI/lib/python3.10/site-packages/torchani/aev.py:16: UserWarning: cuaev not installed
  warnings.warn("cuaev not installed")


Now let's set up a crystal



In [3]:
atoms = Diamond(symbol="C", pbc=True)
print(len(atoms), "atoms in the cell")

8 atoms in the cell


Now let's create a calculator from builtin models:



In [4]:
calculator = torchani.models.ANI1ccx().ase()

/Users/sergiortizropero/miniconda3/envs/ASE_ANI/lib/python3.10/site-packages/torchani/resources/


<div class="alert alert-info"><h4>Note</h4><p>Regardless of the dtype you use in your model, when converting it to ASE
    calculator, it always automatically the dtype to ``torch.float64``. The
    reason for this behavior is, at many cases, the rounding error is too
    large for structure minimization. If you insist on using
    ``torch.float32``, do the following instead:

```python
calculator = torchani.models.ANI1ccx().ase(dtype=torch.float32)</p></div>
```


Now let's set the calculator for ``atoms``:



In [5]:
atoms.set_calculator(calculator)

/var/folders/cq/s30xmbh57xndgmhg1zhwclsh0000gn/T/ipykernel_95390/791988102.py:1: FutureWarning: Please use atoms.calc = calc
  atoms.set_calculator(calculator)


Now let's minimize the structure:



In [6]:
print("Begin minimizing...")
opt = BFGS(atoms)
opt.run(fmax=0.001)
print()

Begin minimizing...
      Step     Time          Energy          fmax
BFGS:    0 10:18:23    -8311.226183        0.000027



Now create a callback function that print interesting physical quantities:



In [7]:
def printenergy(a=atoms):
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))

We want to run MD with constant energy using the Langevin algorithm
with a time step of 1 fs, the temperature 300K and the friction
coefficient to 0.02 atomic units.



In [8]:
dyn = Langevin(atoms, 1 * units.fs, 300 * units.kB, 0.2)
dyn.attach(printenergy, interval=50)

/Users/sergiortizropero/miniconda3/envs/ASE_ANI/lib/python3.10/site-packages/ase/md/md.py:52: FutureWarning: Specify the temperature in K using the 'temperature_K' argument
  warnings.warn(FutureWarning(w))


Now run the dynamics:



In [9]:
print("Beginning dynamics...")
printenergy()
dyn.run(500)

Beginning dynamics...
Energy per atom: Epot = -1038.903eV  Ekin = 0.000eV (T=  0K)  Etot = -1038.903eV
Energy per atom: Epot = -1038.903eV  Ekin = 0.000eV (T=  0K)  Etot = -1038.903eV
Energy per atom: Epot = -1038.881eV  Ekin = 0.028eV (T=220K)  Etot = -1038.853eV
Energy per atom: Epot = -1038.869eV  Ekin = 0.036eV (T=278K)  Etot = -1038.833eV
Energy per atom: Epot = -1038.885eV  Ekin = 0.041eV (T=320K)  Etot = -1038.843eV
Energy per atom: Epot = -1038.876eV  Ekin = 0.025eV (T=197K)  Etot = -1038.850eV
Energy per atom: Epot = -1038.862eV  Ekin = 0.027eV (T=210K)  Etot = -1038.835eV
Energy per atom: Epot = -1038.867eV  Ekin = 0.043eV (T=333K)  Etot = -1038.824eV
Energy per atom: Epot = -1038.867eV  Ekin = 0.022eV (T=170K)  Etot = -1038.845eV
Energy per atom: Epot = -1038.878eV  Ekin = 0.036eV (T=281K)  Etot = -1038.841eV
Energy per atom: Epot = -1038.853eV  Ekin = 0.021eV (T=162K)  Etot = -1038.832eV
Energy per atom: Epot = -1038.867eV  Ekin = 0.036eV (T=276K)  Etot = -1038.831eV


True